# 📘 Task Explanation: Vector Addition Kernel + Grid-Stride Loop + Correctness Verification
## 🎯 1. Objective of the Task

The purpose of this task is to introduce fundamental CUDA programming concepts by implementing a simple parallel operation—vector addition—and then improving it using a grid-stride loop, which is widely used in real production-grade GPU kernels.

You will also verify correctness by comparing GPU output against a CPU reference implementation.

## 🧩 2. Part 1 — Implement a Basic CUDA Vector Addition Kernel
### Task

Write a CUDA kernel where each thread computes one output element:

C[i] = A[i] + B[i]

### Key Concepts

* Computing global thread index:

int idx = blockIdx.x * blockDim.x + threadIdx.x;


* Avoiding out-of-bounds access by checking idx < N.

### Why This Matters

This simple kernel helps you learn:

* How GPUs launch thousands of threads

* How computation is mapped to threads

* CUDA kernel structure (__global__)

* Host-device execution flow

This is the “Hello World” of CUDA and the foundation of all ML acceleration work.

## 🔁 3. Part 2 — Rewrite Using a Grid-Stride Loop
### Task

Modify the kernel so that each thread processes multiple elements using a loop like:

for (int i = idx; i < N; i += blockDim.x * gridDim.x) {
    C[i] = A[i] + B[i];
}

### Why Grid-Stride Loops Are Important

This pattern is used in PyTorch, TensorFlow, cuBLAS, FlashAttention, and most CUDA ML kernels because it allows:

* Handling vectors larger than the number of threads

* Configuring threads independent of data size

* Better GPU utilization and load balancing

* Contiguous memory access for performance

This step marks the transition from “toy kernel” → “real GPU engineering skill.”

## 🧪 4. Part 3 — Verify Correctness
### Task

Create host-side code to:

1. Allocate memory for A, B, and C

2. Initialize A and B with known values

3. Run the GPU kernel

4. Compute a CPU reference result

5. Compare GPU and CPU values

6. Print PASS/FAIL

### What to Check

* All GPU outputs match CPU outputs

* Handle float comparisons with tolerance:

fabs(gpu - cpu) < 1e-6

### Why This Matters

Correctness checking is mandatory in professional GPU development:

* Ensures kernels behave as expected

* Avoids silent numerical bugs

* Builds confidence before performance optimization

Every CUDA kernel in ML frameworks undergoes similar validation.

## 🧠 5. What You Will Learn

By completing this task, you will understand:

Core CUDA Programming Concepts

* Kernel launches

* Thread indexing

* Memory management

* Parallel execution model

High-Performance GPU Design Patterns

* Grid-stride loops

* Coalesced memory access

* Load balancing

Engineering Best Practices

* CPU reference comparison

* GPU debugging workflow

* Repeatable benchmarking

This is foundational knowledge for ML systems engineering, GPU kernel optimization, LLM inference acceleration, and CUDA-based scientific computation.

## 📦 6. Final Deliverables

You should produce the following files:

File	Description
vector_add_basic.cu	Basic per-thread vector addition kernel
vector_add_gridstride.cu	Scalable kernel with grid-stride loop
test_vector_add.cpp/cu	CPU vs GPU correctness checking
benchmark.txt	(Optional) Timing comparison: CPU vs GPU

These can be uploaded to GitHub as part of your Week-1 CUDA project.

In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Fri Dec 12 00:59:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8       

In [ ]:
!apt-get update
!apt-get install -y cuda-toolkit-12-4

Part 1 — Implement a Basic CUDA Vector Addition Kernel

In [ ]:
%%writefile vector_add.cu

#include <iostream>
#include <cuda_runtime.h>
#define N 1024

__global__ void vector_add(float* a, float* b, float* c, int n){
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < n){
        c[id] = a[id] + b[id];
    }
}

int main(){
    float *a, *b, *c;
    size_t bytes = N * sizeof(float);
    cudaMallocManaged(&a, bytes);
    cudaMallocManaged(&b, bytes);
    cudaMallocManaged(&c, bytes);


    for(int i = 0; i < N; i++){
        a[i] = i * 1.0f;
        b[i] = i * 2.0f;
    }


    int threads = 256;
    int blocks  = (N + threads - 1) / threads;
    dim3 grid_dim(blocks);
    dim3 block_dim(threads);

    vector_add<<<grid_dim, block_dim>>>(a, b, c, N);
    cudaDeviceSynchronize();


    std::cout << "GPU output" <<" ";
    for (int j = 0; j < 10; ++j)
        std::cout << c[j] << " ";
     std::cout << std::endl;

    std::cout << "CPU output" <<" ";
    for (int j = 0; j < 10; ++j)
        std::cout << a[j] + b[j] << " ";
     std::cout << std::endl;


    cudaFree(a);
    cudaFree(b);
    cudaFree(c);
    return 0;
}

Overwriting vector_add.cu


In [ ]:
!nvcc -arch=sm_75 vector_add.cu -o vector_add
!./vector_add

GPU output 0 3 6 9 12 15 18 21 24 27 
CPU output 0 3 6 9 12 15 18 21 24 27 


Part 2 — Rewrite Using a Grid-Stride Loop

In [ ]:
%%writefile vector_add_gridstride_skeleton.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>

// --------------------
// Simple CUDA error check macro
// --------------------
#define CUDA_CHECK(call)                                                     \
    do {                                                                     \
        cudaError_t err = (call);                                            \
        if (err != cudaSuccess) {                                            \
            fprintf(stderr, "CUDA error %s:%d: %s\n",                         \
                    __FILE__, __LINE__, cudaGetErrorString(err));            \
            std::exit(EXIT_FAILURE);                                         \
        }                                                                    \
    } while (0)

// --------------------
// TODO: Implement grid-stride vector add kernel
// Requirements:
//  - Use global thread index: tid = blockIdx.x * blockDim.x + threadIdx.x
//  - Use stride = blockDim.x * gridDim.x
//  - Loop: for (i = tid; i < N; i += stride) C[i] = A[i] + B[i]
//  - A, B are read-only; C is output
// --------------------
__global__ void vectorAddGridStride(const float* A, const float* B, float* C, int N) {
    // TODO: compute tid
    // TODO: compute stride
    // TODO: grid-stride loop
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    if(tid < N){
        for (int i = tid; i < N; i += stride){
            C[i] = A[i] + B[i];
        }
    }
}

// --------------------
// CPU reference for correctness check
// --------------------
static void vectorAddCPU(const float* A, const float* B, float* C, int N) {
    for (int i = 0; i < N; i++) C[i] = A[i] + B[i];
}

// --------------------
// TODO: Fill helper to compare results
// Requirements:
//  - Return true if all elements match within tolerance (e.g., 1e-6)
//  - Print the first mismatch (index + values) then return false
// --------------------
static bool checkClose(const float* gpu, const float* cpu, int N, float tol) {
    // TODO
    for (int i = 0; i < N; i++){
        float diff = fabsf(gpu[i] - cpu[i]);
        if (diff > tol){
          printf("Mismatch at %d: gpu=%f cpu=%f diff=%f\n", i, gpu[i], cpu[i], diff);
          return false;
        }

    }

    return true;
}

int main() {
    // --------------------
    // Config
    // --------------------
    const int N = 1 << 24;          // ~16 million floats
    const size_t bytes = size_t(N) * sizeof(float);
    const float tol = 1e-6f;

    // --------------------
    // Host alloc
    // --------------------
    float* hA = (float*)std::malloc(bytes);
    float* hB = (float*)std::malloc(bytes);
    float* hC_gpu = (float*)std::malloc(bytes);
    float* hC_cpu = (float*)std::malloc(bytes);

    if (!hA || !hB || !hC_gpu || !hC_cpu) {
        fprintf(stderr, "Host malloc failed.\n");
        return EXIT_FAILURE;
    }

    // --------------------
    // Init host data
    // --------------------
    for (int i = 0; i < N; i++) {
        hA[i] = 0.001f * i;
        hB[i] = 0.002f * i;
    }

    // --------------------
    // Device alloc
    // --------------------
    float *dA = nullptr, *dB = nullptr, *dC = nullptr;
    CUDA_CHECK(cudaMalloc(&dA, bytes));
    CUDA_CHECK(cudaMalloc(&dB, bytes));
    CUDA_CHECK(cudaMalloc(&dC, bytes));

    // --------------------
    // H2D copy
    // --------------------
    CUDA_CHECK(cudaMemcpy(dA, hA, bytes, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(dB, hB, bytes, cudaMemcpyHostToDevice));

    // --------------------
    // TODO: Choose launch config
    // Requirements:
    //  - Set blockSize (commonly 128/256/512)
    //  - Set gridSize (e.g., (N + blockSize - 1)/blockSize) OR a fixed large grid
    // --------------------
    int blockSize = 256; // TODO
    int gridSize  = (N + blockSize - 1)/blockSize; // TODO

    // --------------------
    // Launch kernel
    // --------------------
    vectorAddGridStride<<<gridSize, blockSize>>>(dA, dB, dC, N);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    // --------------------
    // D2H copy
    // --------------------
    CUDA_CHECK(cudaMemcpy(hC_gpu, dC, bytes, cudaMemcpyDeviceToHost));

    // --------------------
    // CPU reference
    // --------------------
    vectorAddCPU(hA, hB, hC_cpu, N);

    // --------------------
    // TODO: Correctness check
    // --------------------
    bool ok = checkClose(hC_gpu, hC_cpu, N, tol); // TODO: make checkClose work
    printf("Correctness: %s\n", ok ? "PASS" : "FAIL");

    // --------------------
    // Cleanup
    // --------------------
    CUDA_CHECK(cudaFree(dA));
    CUDA_CHECK(cudaFree(dB));
    CUDA_CHECK(cudaFree(dC));
    std::free(hA);
    std::free(hB);
    std::free(hC_gpu);
    std::free(hC_cpu);

    return ok ? EXIT_SUCCESS : EXIT_FAILURE;
}

Overwriting vector_add_gridstride_skeleton.cu


In [ ]:
!nvcc -arch=sm_75 vector_add_gridstride_skeleton.cu -o vector_add_gridstride_skeleton
!./vector_add_gridstride_skeleton

Correctness: PASS
